In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

articles = pd.read_csv("/kaggle/input/h-and-m-personalized-fashion-recommendations/articles.csv")
transactions = pd.read_csv("/kaggle/input/h-and-m-personalized-fashion-recommendations/transactions_train.csv")

articles.drop(['product_code','product_type_no','prod_name','product_type_name','graphical_appearance_no','graphical_appearance_name','colour_group_code','colour_group_name','index_group_name'],axis=1,inplace = True)

merge_train = pd.merge(articles, transactions, on="article_id")

In [2]:
merge_train_y = merge_train[merge_train["t_dat"]>="2020-09-10"].copy()
merge_train_x = merge_train.drop(merge_train[merge_train.t_dat >= "2020-09-10"].index)

In [3]:
merge_train_y

In [4]:
merge_train_x

In [5]:
merge_train_x.drop(['article_id','perceived_colour_value_id','perceived_colour_value_name','perceived_colour_master_id','perceived_colour_master_name','department_no','department_name','index_code','index_name','index_group_no','section_no','section_name','garment_group_no','garment_group_name','detail_desc','t_dat','price','sales_channel_id'],axis=1,inplace = True)
merge_train_y.drop(['article_id','perceived_colour_value_id','perceived_colour_value_name','perceived_colour_master_id','perceived_colour_master_name','department_no','department_name','index_code','index_name','index_group_no','section_no','section_name','garment_group_no','garment_group_name','detail_desc','t_dat','price','sales_channel_id'],axis=1,inplace = True)

In [6]:
merge_train_x_rated = merge_train_x.assign(weight=1).copy()
merge_train_y_rated = merge_train_y.assign(weight=1).copy()

In [9]:
from surprise import Dataset
from surprise import Reader

In [10]:
merge_train_x_grouped = merge_train_x_rated.groupby(["customer_id","product_group_name"]).sum()["weight"].reset_index()
merge_train_y_grouped = merge_train_y_rated.groupby(["customer_id","product_group_name"]).sum()["weight"].reset_index()

In [11]:
merge_train_x_grouped

In [12]:
merge_train_y_grouped

In [ ]:
merge_train_x_grouped["weight"].max()

In [13]:
reader = Reader(rating_scale=(1, merge_train_x_grouped["weight"].max()))

In [14]:
data = Dataset.load_from_df(merge_train_x_grouped[["customer_id", "product_group_name", "weight"]], reader)

In [15]:
trainingSet = data.build_full_trainset()

In [16]:
from surprise import KNNWithMeans

# To use item-based cosine similarity
sim_options = {
    "name": "cosine",
    "user_based": False,  # Compute  similarities between items
}
algo = KNNWithMeans(sim_options=sim_options)

In [17]:
algo.fit(trainingSet)

In [ ]:
prediction = algo.predict('00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657', 'Garment Full body')

In [ ]:
prediction.est

In [ ]:
merge_train_grouped[merge_train_grouped["customer_id"]=='00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657']

In [18]:
cat_arr = np.array(pd.Categorical(merge_train_x_grouped['product_group_name']).categories)

In [ ]:
cat_arr

In [49]:
for cat in cat_arr:
    prediction = algo.predict(merge_train_y_grouped.at[209880,'customer_id'], cat)
    print("Prediction pour " + cat +" est : ")
    print(prediction.est)
    print("valeur x :")
    x = merge_train_x_grouped.loc[(merge_train_x_grouped["customer_id"]==merge_train_y_grouped.at[209880,'customer_id'])&(merge_train_x_grouped['product_group_name']==cat)].weight
    if x.empty:
        print(0)
    else :
        print(x.values[0])
    print("valeur y :")
    y = merge_train_y_grouped.loc[(merge_train_y_grouped["customer_id"]==merge_train_y_grouped.at[209880,'customer_id'])&(merge_train_y_grouped['product_group_name']==cat)].weight
    if y.empty:
        print(0)
    else :
        print(y.values[0])